In [9]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time as t
import requests
import csv
from selenium import webdriver
from pprint import pprint
from scraper import scrape_site
from pathlib import Path
import pandas as pd
from config import subscription_key
from urllib.error import HTTPError
from googleapi import google
from urllib.parse import urlencode, urlparse, parse_qs, urlsplit

import requests.exceptions
from collections import deque

from lxml.html import fromstring
from requests import get

In [17]:
file_to_output = Path("Data_Scraped/Contacts_Final.csv")

In [16]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', headless=False, **executable_path)

In [18]:
url = 'https://www.bizjournals.com/houston/potm?l=&time=&type=&ind=17'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Final_List = []
dict_list =[]
links = []

In [28]:

for link in soup.find_all("a", class_="item", href=True):
        links.append(link.get('href'))
#Get Next button
for link in soup.find_all("a", class_="btn btn--primary pull-right", href=True):
    button = link.get('href')


for i in links:
    if '/houston/potmsearch/detail/submission' in i:
        d = {}
        link = "https://www.bizjournals.com/" + i
        newI = i.split("?")
        slicefront = (newI[0])
        name = slicefront[46:]
        final_name = name.replace('_', " ")
        d["Name"] = final_name
        d["Link"] = link
        if d not in dict_list:
            dict_list.append(d)
        else:
            pass
    else:
        pass

In [29]:

try:
#     browser.click_link_by_partial_href('Next')
    browser.click_link_by_href(button) 
#     browser.find_link_by_text('Next').first.click()
#     browser.links.find_by_partial_text("Next").click()
    print("Went to the next page")
    html = ''
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
except:
    print("Finished all pages")


Finished all pages


In [30]:
len(dict_list)

99

In [31]:
pprint(dict_list)

[{'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Name': 'Charles Harrell'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489778/Bryce_Kaufman?l=&time=&ind=17&type=&ro=1',
  'Name': 'Bryce Kaufman'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489777/Kevin_Richardson?l=&time=&ind=17&type=&ro=2',
  'Name': 'Kevin Richardson'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489683/Zach_Burford?l=&time=&ind=17&type=&ro=3',
  'Name': 'Zach Burford'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489678/Laura_S_Rahman?l=&time=&ind=17&type=&ro=4',
  'Name': 'Laura S Rahman'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489465/Daniel_Dutko?l=&time=&ind=17&type=&ro=5',
  'Name': 'Daniel Dutko'},
 {'Link': 'https://www.bizjournals.com//houston/potmsear

In [32]:
browser.visit("https://www.bizjournals.com/houston/potmsearch/detail/submission/6488386/Charles_M_Rosson?time=most_recent&ind=86&type=all&id=&ro=1")
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
date_info= soup.find_all('a', class_='meta-item')
print(date_info[0].text)


Promotion



In [33]:
for i in dict_list:

        try:
                browser.visit(i["Link"])
                t.sleep(1)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                info_list = soup.find_all('p', class_ = 'cXenseParse')
                date_info = soup.find_all('span', class_='meta-item')
                type_info= soup.find_all('a', class_='meta-item')
                try: 
                    i["Title"] = info_list[0].text
                    i["Blurb"] = info_list[1].text
                    i["Publish Date"] = date_info[0].text
                    i["New Hire/Promotion"] = type_info[0].text
                except:
                    print(f"Missing info for {i['Name']}")
                Final_List.append(i)

        except HTTPError as error:
            print(error)
length_Final_List = len(Final_List)
print(f"Finished pulling. {length_Final_List} items pulled")

Missing info for Randy SorrelsPresident State Bar of Texas
Finished pulling. 99 items pulled


In [34]:
len(Final_List)

99

In [35]:
pprint(Final_List)

[{'Blurb': 'Corporate attorney Charles Harrell has joined the Corporate & '
           'Securities practice in the Houston office of Jackson Walker LLP. '
           'With decades of experience, he has guided businesses ranging from '
           'technology, healthcare, and energy startups to mid-market\n'
           'and Fortune 100 companies through all types of corporate matters. '
           'His focus is on\n'
           'complex transactions including mergers & acquisitions, '
           'cross-border matters, finance and capital markets. He also advises '
           'board committees on corporate governance.',
  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Name': 'Charles Harrell',
  'New Hire/Promotion': '\nNew Hire\n',
  'Publish Date': 'Nov 27, 2020',
  'Title': 'Partner at Jackson Walker LLP'},
 {'Blurb': 'Bryce D. Kaufman has been promoted to counsel at Latham & Watkins '
           'in Ho

  'Publish Date': 'Nov 27, 2020',
  'Title': 'Trial Associate at Copeland & Rice'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6470456/Randy_SorrelsPresident_State_Bar_of_Texas?l=&time=&ind=17&type=&ro=90',
  'Name': 'Randy SorrelsPresident State Bar of Texas',
  'Title': 'Randy Sorrels has been sworn in as the 2019 – 2020 President of '
           'the State Bar of Texas, the third largest bar in the United '
           'States. As the first president from Houston in 12 years, Mr. '
           'Sorrels is also the managing partner of the personal injury and '
           'business litigation law firm of Abraham, Watkins, Nichols, '
           'Sorrels, Agosto & Aziz. He is double board certified in Personal '
           'Injury Trial Law and Civil Trial Law by the Texas Board of Legal '
           'Specialization and has been selected as one of the top 100 lawyers '
           'in Texas by the Texas Super Lawyers publication for the last 12 '
         

In [36]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",\nNew Hire\n
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",\nPromotion\n
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",\nPromotion\n
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",\nNew Hire\n
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",\nNew Hire\n


In [37]:
FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",New Hire
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",Promotion
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",Promotion
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",New Hire
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",New Hire


In [38]:
pd.set_option('display.max_rows', None)
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",New Hire
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",Promotion
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",Promotion
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",New Hire
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",New Hire
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"Nov 27, 2020",Promotion
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"Nov 27, 2020",New Hire
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","Nov 27, 2020",New Hire
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"Nov 27, 2020",New Hire
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"Nov 27, 2020",New Hire


In [39]:
FL.isna().sum()

Name                  0
Link                  0
Title                 0
Blurb                 1
Publish Date          1
New Hire/Promotion    1
dtype: int64

In [40]:
file_to_output = Path("Data_Scraped/Contacts_Final.csv")

In [41]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

## Searching Bing for website and emails

In [42]:
assert subscription_key


In [43]:
# Set the URL to bing API
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [ ]:
search_term = "Bethany Spare Associate at Yetter Coleman LLP"

In [ ]:
headers = {"Ocp-Apim-Subscription-Key": "5158c76dee144128862504b6b30ae70a"}
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
search_results

In [ ]:
search_results['webPages']['value'][0]['url']

In [45]:
for i in Final_List:
    try:
        search_url = "https://api.bing.microsoft.com/v7.0/search"
        search_term = i["Name"] + ' ' + i["Title"]
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
        print(f"Found {i['Name']}!")
    except:
        print(f"Could not find {i['Name']}")
    t.sleep(1)

Found Charles Harrell!
Found Bryce Kaufman!
Found Kevin Richardson!
Found Zach Burford!
Found Laura S Rahman!
Found Daniel Dutko!
Found Bethany Spare!
Found Andrea Hellyer!
Found Adrienne Randle Bond!
Found Eric Thiergod Sr!
Found Charles M Rosson!
Found Heath Trisdale!
Found Michael Hamilton!
Found Travis Clardy!
Found David Peterman!
Found Jonathan Newton!
Found John Farrell!
Found Connie Pfeiffer!
Found Kathlyn Hufstetler!
Found Sammy Ford IV!
Found Colleen Migl!
Found Brooks Tueting!
Found Edward Ed L Ripley!
Found Stephanie McGraw!
Found Ben Walther!
Found Michael L Telford!
Found Elizabeth Liz Webb!
Found Caroline C Pace!
Found Keith A Taylor!
Found Gabriela Gabby M Barake!
Found Stephen Jacobson!
Found Justo Mendez!
Found Samuel Louis!
Found S Priya Coffey!
Found Joe Flack!
Found Christopher Hogan!
Found Samantha Thompson!
Found David W Parker!
Found Kristhy Peguero!
Found William AI Trey McDonald III!
Found Briggs Knight!
Found Kate Kaufmann Shih!
Found Lindsey Raspino!
Found K

In [46]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Bing Search Link
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",\nNew Hire\n,https://www.jw.com/news/welcome-charles-harrel...
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/bryce-kaufman
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/kevin-richardson
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",\nNew Hire\n,https://reynoldsfrizzell.com/attorneys/zach-bu...
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",\nNew Hire\n,https://www.keanmiller.com/laura-s-rahman.html


In [47]:
FL.isna().sum()

Name                  0
Link                  0
Title                 0
Blurb                 1
Publish Date          1
New Hire/Promotion    1
Bing Search Link      0
dtype: int64

In [48]:
FL.Title.value_counts()

Partner at Jackson Walker LLP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               7
Associate at Yetter Coleman LLP                                                                                                                                                                           

In [49]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

In [ ]:
#Convert Bing Links to a list
# BSL = FL["Bing Search Link"].to_list()
# BSL

### Testing the email search

In [ ]:
# # starting url. replace google with your own url.
# starting_url = 'https://www.kslaw.com/people?locale=en&office_id=4'

# # a queue of urls to be crawled
# unprocessed_urls = deque([starting_url])

# # set of already crawled urls for email
# processed_urls = set()

# # a set of fetched emails
# emails = set()

# # process urls one by one from unprocessed_url queue until queue is empty
# while len(unprocessed_urls):

#     # move next url from the queue to the set of processed urls
#     url = unprocessed_urls.popleft()
#     processed_urls.add(url)

#     # extract base url to resolve relative links
#     parts = urlsplit(url)
#     base_url = "{0.scheme}://{0.netloc}".format(parts)
#     path = url[:url.rfind('/')+1] if '/' in parts.path else url

#     # get url's content
#     print("Crawling URL %s" % url)
#     try:
#         response = requests.get(url)
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         continue

#     # extract all email addresses and add them into the resulting set
#     # You may edit the regular expression as per your requirement
#     new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#     emails.update(new_emails)
#     print(emails)
#     # create a beutiful soup for the html document
#     soup = BeautifulSoup(response.text, 'lxml')
#     # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
#     for anchor in soup.find_all("a"):
#         # extract link url from the anchor
#         link = anchor.attrs["href"] if "href" in anchor.attrs else ''
#         # resolve relative links (starting with /)
#         if link.startswith('/'):
#             link = base_url + link
#         elif not link.startswith('http'):
#             link = path + link
#         # add the new url to the queue if it was not in unprocessed list nor in processed list yet
#         if not link in unprocessed_urls and not link in processed_urls:
#             unprocessed_urls.append(link)

In [ ]:
for i in Final_List:
#     print(i["Bing Search Link"])
    response = requests.get(i["Bing Search Link"])
    print(response)

In [50]:
for i in Final_List:
    emails = set()
    try:
        response = requests.get(i["Bing Search Link"])
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
        emails.update(new_emails)
        i['Bing Emails'] = emails
        if emails == set():
            print(f'Set empty on {i["Name"]}')
            emails = set()
        else:
            print(f"Successfully captured an email for {i['Name']}")
            emails = set()
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors and continue with next url
        print(f"Could not find {i['Name']}")
        continue
print(f"Finished all email grabs")

Set empty on Charles Harrell
Successfully captured an email for Bryce Kaufman
Successfully captured an email for Kevin Richardson
Successfully captured an email for Zach Burford
Set empty on Laura S Rahman
Successfully captured an email for Daniel Dutko
Set empty on Bethany Spare
Set empty on Andrea Hellyer
Successfully captured an email for Adrienne Randle Bond
Set empty on Eric Thiergod Sr
Set empty on Charles M Rosson
Successfully captured an email for Heath Trisdale
Set empty on Michael Hamilton
Set empty on Travis Clardy
Successfully captured an email for David Peterman
Set empty on Jonathan Newton
Set empty on John Farrell
Set empty on Connie Pfeiffer
Set empty on Kathlyn Hufstetler
Set empty on Sammy Ford IV
Set empty on Colleen Migl
Set empty on Brooks Tueting
Set empty on Edward Ed L Ripley
Set empty on Stephanie McGraw
Set empty on Ben Walther
Successfully captured an email for Michael L Telford
Successfully captured an email for Elizabeth Liz Webb
Successfully captured an em

In [59]:
Final_List

[{'Name': 'Charles Harrell',
  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Title': 'Partner at Jackson Walker LLP',
  'Blurb': 'Corporate attorney Charles Harrell has joined the Corporate & Securities practice in the Houston office of Jackson Walker LLP. With decades of experience, he has guided businesses ranging from technology, healthcare, and energy startups to mid-market\nand Fortune 100 companies through all types of corporate matters. His focus is on\ncomplex transactions including mergers & acquisitions, cross-border matters, finance and capital markets. He also advises board committees on corporate governance.',
  'Publish Date': 'Nov 27, 2020',
  'New Hire/Promotion': '\nNew Hire\n',
  'Bing Search Link': 'https://www.jw.com/news/welcome-charles-harrell-houston-corporate-securities-partner/',
  'Bing Emails': set()},
 {'Name': 'Bryce Kaufman',
  'Link': 'https://www.bizjournals.com//houston

In [66]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Bing Search Link,Bing Emails
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",\nNew Hire\n,https://www.jw.com/news/welcome-charles-harrel...,{}
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/bryce-kaufman,{bryce.kaufman@lw.com}
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/kevin-richardson,{kevin.richardson@lw.com}
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",\nNew Hire\n,https://reynoldsfrizzell.com/attorneys/zach-bu...,{zburford@reynoldsfrizzell.com}
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",\nNew Hire\n,https://www.keanmiller.com/laura-s-rahman.html,{}


In [69]:
FL['Bing Emails'] = FL['Bing Emails'].astype(str)

In [72]:
FL["Bing Emails"] = FL["Bing Emails"].str.strip('{}')
FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')


In [73]:
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Bing Search Link,Bing Emails
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",New Hire,https://www.jw.com/news/welcome-charles-harrel...,set()
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",Promotion,https://www.lw.com/people/bryce-kaufman,'bryce.kaufman@lw.com'
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",Promotion,https://www.lw.com/people/kevin-richardson,'kevin.richardson@lw.com'
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",New Hire,https://reynoldsfrizzell.com/attorneys/zach-bu...,'zburford@reynoldsfrizzell.com'
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",New Hire,https://www.keanmiller.com/laura-s-rahman.html,set()
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"Nov 27, 2020",Promotion,https://www.rustyhardin.com/our-team/daniel-du...,"'ddutko@rustyhardin.com', 'info@rustyhardin.com'"
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"Nov 27, 2020",New Hire,https://www.yettercoleman.com/profiles/spare-b...,set()
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","Nov 27, 2020",New Hire,https://www.bizjournals.com/houston/potmsearch...,set()
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"Nov 27, 2020",New Hire,https://www.craincaton.com/attorneys/adrienne-...,"'info@craincaton.com', 'abond@craincaton.com'"
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"Nov 27, 2020",New Hire,https://www.keanmiller.com/eric-r-thiergood-sr...,set()


In [74]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

In [ ]:
Final_List